In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [8]:
df = pd.read_csv('cleaned_dataset.csv', encoding='ISO-8859-1')
df.head()


,doi,polymer(s),is_solvent_blend,solvent(s),solvent_components,solution_concentration,solution_concentration_unit,needle_type,needle_diameter_g,collector_type,rotation_speed_rpm,voltage_kv,flow_rate_ml/h,tip_collector_distance_cm,temperature_c,humidity_%,was_formation_stable,fiber_diameter_nm,fiber_diameter_variation_nm
0,https://doi.org/10.1016/j.jafr.2019.100015,CA,False,ACETONE,"[{""solventName"": ""ACETONE"", ""weight"": null, ""w...",15.0,wt%,Single Needle,22.0,Flat,NaN,12.0,2.0,6.0,23.0,45,True,1346.0,812.0
1,https://doi.org/10.1016/j.jafr.2019.100015,CA,False,ACETONE,"[{""solventName"": ""ACETONE"", ""weight"": null, ""w...",12.0,wt%,Single Needle,22.0,Flat,NaN,16.0,2.0,6.0,23.0,45,True,488.0,221.0
2,https://doi.org/10.1016/j.jafr.2019.100015,CA,False,ACETONE,"[{""solventName"": ""ACETONE"", ""weight"": null, ""w...",15.0,wt%,Single Needle,22.0,Flat,NaN,16.0,2.0,10.0,23.0,45,True,823.0,519.0
3,https://doi.org/10.1016/j.jafr.2019.100015,CA,False,ACETONE,"[{""solventName"": ""ACETONE"", ""weight"": null, ""w...",9.0,wt%,Single Needle,22.0,Flat,NaN,12.0,2.0,6.0,23.0,45,True,452.0,236.0
4,https://doi.org/10.1016/j.jafr.2019.100015,CA,False,ACETONE,"[{""solventName"": ""ACETONE"", ""weight"": null, ""w...",12.0,wt%,Single Needle,22.0,Flat,NaN,12.0,2.0,10.0,23.0,45,True,801.0,382.0


In [9]:
polymers = [
    "PVDF", "PVA", "PVP", "PAN", "PS", "PCL", "PMMA", "PLA", "Y_PGA", "PDLLA",
    "CA", "AROMATIC_PI", "PET", "GELATIN", "PA6", "HA", "PEO", "PVC", "NYLON_6",
    "PSA", "SILK", "PVAC", "TPU", "COLLAGEN"
]

In [10]:
def detect_outliers_iqr(series):
    numeric_series = pd.to_numeric(series, errors='coerce').dropna()
    if numeric_series.empty:
        return pd.Series([False] * len(series), index=series.index)
    
    Q1 = numeric_series.quantile(0.25)
    Q3 = numeric_series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    numeric_full = pd.to_numeric(series, errors='coerce')
    return (numeric_full < lower_bound) | (numeric_full > upper_bound)

df['tail_observation'] = detect_outliers_iqr(df['fiber_diameter_nm'])

print(df[['fiber_diameter_nm', 'tail_observation']].head(20))

    fiber_diameter_nm  tail_observation
0              1346.0              True
1               488.0             False
2               823.0             False
3               452.0             False
4               801.0             False
5               749.0             False
6               666.0             False
7               977.0             False
8               404.0             False
9              1104.0             False
10              562.0             False
11             1149.0             False
12              478.0             False
13              756.0             False
14              653.0             False
15              120.0             False
16              197.0             False
17              218.0             False
18              221.0             False
19              225.0             False


In [11]:
df

,doi,polymer(s),is_solvent_blend,solvent(s),solvent_components,solution_concentration,solution_concentration_unit,needle_type,needle_diameter_g,collector_type,rotation_speed_rpm,voltage_kv,flow_rate_ml/h,tip_collector_distance_cm,temperature_c,humidity_%,was_formation_stable,fiber_diameter_nm,fiber_diameter_variation_nm,tail_observation
0,https://doi.org/10.1016/j.jafr.2019.100015,CA,False,ACETONE,"[{""solventName"": ""ACETONE"", ""weight"": null, ""w...",15.0,wt%,Single Needle,22.0,Flat,NaN,12.0,2.0,6.0,23.0,45,True,1346.0,812.0,True
1,https://doi.org/10.1016/j.jafr.2019.100015,CA,False,ACETONE,"[{""solventName"": ""ACETONE"", ""weight"": null, ""w...",12.0,wt%,Single Needle,22.0,Flat,NaN,16.0,2.0,6.0,23.0,45,True,488.0,221.0,False
2,https://doi.org/10.1016/j.jafr.2019.100015,CA,False,ACETONE,"[{""solventName"": ""ACETONE"", ""weight"": null, ""w...",15.0,wt%,Single Needle,22.0,Flat,NaN,16.0,2.0,10.0,23.0,45,True,823.0,519.0,False
3,https://doi.org/10.1016/j.jafr.2019.100015,CA,False,ACETONE,"[{""solventName"": ""ACETONE"", ""weight"": null, ""w...",9.0,wt%,Single Needle,22.0,Flat,NaN,12.0,2.0,6.0,23.0,45,True,452.0,236.0,False
4,https://doi.org/10.1016/j.jafr.2019.100015,CA,False,ACETONE,"[{""solventName"": ""ACETONE"", ""weight"": null, ""w...",12.0,wt%,Single Needle,22.0,Flat,NaN,12.0,2.0,10.0,23.0,45,True,801.0,382.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,https://doi.org/10.3139/217.3278,PVA,False,WATER,"[{""solventName"": ""DISTILLED_WATER"", ""weight"": ...",10.0,wt%,Single Needle,22.0,Flat,NaN,16.0,0.5,18.0,25.0,NaN,True,636.0,76.0,False
805,https://doi.org/10.3139/217.3278,PVA,False,WATER,"[{""solventName"": ""DISTILLED_WATER"", ""weight"": ...",10.0,wt%,Single Needle,22.0,Flat,NaN,14.0,1.0,15.0,25.0,NaN,True,659.0,62.0,False
806,https://doi.org/10.3139/217.3278,PVA,False,WATER,"[{""solventName"": ""DISTILLED_WATER"", ""weight"": ...",8.0,wt%,Single Needle,22.0,Flat,NaN,18.0,0.5,15.0,25.0,NaN,True,411.0,73.0,False
807,https://doi.org/10.3139/217.3278,PVA,False,WATER,"[{""solventName"": ""DISTILLED_WATER"", ""weight"": ...",8.0,wt%,Single Needle,22.0,Flat,NaN,16.0,1.0,10.0,25.0,NaN,True,456.0,83.0,False


In [12]:
df.to_csv('Dataset.csv', index=False)